In [2]:
'''
===========================
nympha_ellipse
===========================

Demonstrates a flower of nymphaea as 3D surface(ellipsoid) objects.

'''

from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import Circle
import mpl_toolkits.mplot3d.art3d as art3d
import matplotlib.pyplot as plt
import numpy as np
import csv
import os

# setting organ
class Organ :
	def __init__(self, rx, ry, rz, xdeg, zdeg, zpo, hb, col, reso):

		self.rx = rx #minor length
		self.ry = ry #major length
		self.rz = rz 

		self.reso = reso #surfaces resolution

#organ's shape as ellipsoid surface
		u = np.linspace(0, 2 * np.pi, reso)
		v = np.linspace(0, 1/2*np.pi, reso)
		x = self.rx * np.outer(np.cos(u), np.sin(v))
		y = self.ry * np.outer(np.sin(u), np.sin(v))
		z = -self.rz * np.outer(np.ones(np.size(u)), np.cos(v))
		xyz = np.dstack((x, y, z))

#rotate & transfer organ
		self.xdeg = xdeg
		xrot = np.array([[1, 0, 0],
						[0, np.cos(xdeg), -1*np.sin(xdeg)],
						[0, np.sin(xdeg), np.cos(xdeg)]])
		self.zdeg = zdeg
		zrot = np.array([[np.cos(zdeg), -1*np.sin(zdeg), 0],
						[np.sin(zdeg), np.cos(zdeg), 0],
						[0, 0, 1]])

		for i in range(reso):
			for j in range(reso):
				xyz[i,j,:] = np.dot(xrot, xyz[i,j,:].T)
				xyz[i,j,:] = xyz[i,j,:] + [0, self.ry * np.cos(xdeg) + db, self.ry * np.sin(xdeg) + zpo]
				xyz[i,j,:] = np.dot(zrot, xyz[i,j,:].T)
				if record == True:
					csvWriter.writerow(xyz[i,j,:]) #record in csv

		self.col = col

##########################################################################################

#setting parameter of each organ

def orgforms(dx1 = 0.040, dx2 = 0.001, x0 = 10, xt = 2.0, txi = 20, dy1 = 0.020, dy2 = 0.004, y0= 35, yt = 12.0, tyi = 20):
#parameter to set size of ellipsoid

	rxs = np.linspace(0, n_organ-1, n_organ) # minor length
	rxs[0:txi] = x0 - x0 * dx1 * rxs[0:txi] # x = x0 - ai and  a = x0*dx1
	rxs[txi:n_organ] = xt - x0 * dx2 * (rxs[txi:n_organ] - txi) # x = xt - b(i-tx) and b = x0*dx2 and tx = txi-1
    
	rys = np.linspace(0, n_organ-1, n_organ) # major length
	rys[0:tyi] = y0 - y0 * dy1 * rys[0:tyi] # y = y0 - ai and  a = y0*dy1
	rys[tyi:n_organ] = yt - y0 * dy2 * (rys[tyi:n_organ] - tyi) # y = yt - b(i-ty) and b = y0*dy2 and ty = tyi-1

	#the point finishing transition
	tmax = max([txi, tyi]) 

	#organs swell
	rzs = np.zeros(n_organ)
	for i in range(n_organ):
		if i < tmax:
			rzs[i] = 0 #mode1
		else:
			rzs[i] = 0 #mode2

	return (rxs, rys, tmax, rzs)

####################################################################
#parameter to elevate each organ

def orgposi(pitch = -0.01):

#parameter to rotate each organ along z-axis
	zdegs = np.zeros(n_organ)
	zdegs[0] = 0
	zdegs[1] = np.pi
	zdegs[2] = np.pi * 3/2
	zdegs[3] = np.pi * 1/2
	for i in range(4, n_organ):
		zdegs[i] = zdegs[i - 1] + 137.5/180 *  np.pi

# height where each organ is put 
	zpos = np.zeros(n_organ)
	dzps = np.exp(pitch * zdegs)
	zpos[0] = 0
	zpos[1] = 0
	zpos[2] = 0
	zpos[3] = 0
	for i in range (1, n_organ):
		zpos[i] = zpos[i-1] + dzps[i]
	zpos = zpos/max(zpos) * hb

	return (zdegs, zpos)

########################################################################

def elevangle(max_ang = np.pi / 4, min_ang = np.pi / 4):

	xdegs = np.zeros(n_organ)
	xdegs[0:4] = min_ang
	for i in range(4, n_organ):
		if i < tmax:
			xdegs[i] = xdegs[i - 1] + (max_ang - min_ang) / (tmax-4)
		else:
			xdegs[i] = 90/180 * np.pi

	return xdegs
##########################################################################################
#setting

n_organ = 100
reso = 90

db = 10 #half of Base' s Diameter
hb = 5 #Height of Base
bu = np.linspace(0, 2 * np.pi, reso)

#upper part of base
bux = np.outer(db * np.cos(bu), np.ones(np.size(bu)))
buy = np.outer(db * np.sin(bu), np.ones(np.size(bu)))
buz = np.outer(np.ones(np.size(bu)), np.linspace(0, hb, reso))

#uppermost part of base
upp = Circle((0, 0), db, color = "red")

#basal part of base
bv = np.linspace(0, 1/2*np.pi, reso)
bbx = -db * np.outer(np.cos(bu), np.sin(bv))
bby = -db * np.outer(np.sin(bu), np.sin(bv))
bbz = -db * np.outer(np.ones(np.size(bu)), np.cos(bv))

##########################################################################################

#csv file
#Trueでファイルに書き出し
os.chdir('D:/theoretical_waterlily_data/csv/ea') #データが大きいのでハードディスクに保管
record = True
namehead = "nympha-ell_ea_20180530"

for curr_maxang in range(0, 10, 1):
    for curr_minang in range(0, 10, 1):
        namebody = '_' + str(int(curr_minang)).zfill(2) + '-' + str(int(curr_maxang)).zfill(2)
        
        if record == True:
            csv_coo = open(namehead + namebody + '.csv', 'x', newline='')
            csvWriter = csv.writer(csv_coo)
            for i in range(reso):
                for j in range(reso):
                    if curr_maxang >= curr_minang:
                        csvWriter.writerow([bux[i,j], buy[i,j], buz[i,j]])
                        csvWriter.writerow([bbx[i,j], bby[i,j], bbz[i,j]])
                    else: #条件に合わない花形態の座標情報はNaN扱い
                        csvWriter.writerow([np.nan, np.nan, np.nan])
        
        #draw each organ
        rxs, rys, tmax, rzs = orgforms()
        zdegs, zpos = orgposi()
        xdegs = elevangle(max_ang = np.pi/18 * curr_maxang, min_ang = np.pi/18 * curr_minang) 

        if curr_maxang >= curr_minang:
            for n in range(n_organ):
                rx = rxs[n]
                ry = rys[n]
                rz = rzs[n]
                xdeg = xdegs[n]
                zdeg = zdegs[n]
                zpo = zpos[n]
                col = "red"
                organ = Organ(rx, ry, rz, xdeg, zdeg, zpo, hb, col, reso)
        else:
            csvWriter.writerow([np.nan, np.nan, np.nan])
                
        if record == True:
            csv_coo.close()
            print(namehead + namebody + '.csv finished')

print("completed")

nympha-ell_ea_20180530_00-00.csv finished
nympha-ell_ea_20180530_01-00.csv finished
nympha-ell_ea_20180530_02-00.csv finished
nympha-ell_ea_20180530_03-00.csv finished
nympha-ell_ea_20180530_04-00.csv finished
nympha-ell_ea_20180530_05-00.csv finished
nympha-ell_ea_20180530_06-00.csv finished
nympha-ell_ea_20180530_07-00.csv finished
nympha-ell_ea_20180530_08-00.csv finished
nympha-ell_ea_20180530_09-00.csv finished
nympha-ell_ea_20180530_00-01.csv finished
nympha-ell_ea_20180530_01-01.csv finished
nympha-ell_ea_20180530_02-01.csv finished
nympha-ell_ea_20180530_03-01.csv finished
nympha-ell_ea_20180530_04-01.csv finished
nympha-ell_ea_20180530_05-01.csv finished
nympha-ell_ea_20180530_06-01.csv finished
nympha-ell_ea_20180530_07-01.csv finished
nympha-ell_ea_20180530_08-01.csv finished
nympha-ell_ea_20180530_09-01.csv finished
nympha-ell_ea_20180530_00-02.csv finished
nympha-ell_ea_20180530_01-02.csv finished
nympha-ell_ea_20180530_02-02.csv finished
nympha-ell_ea_20180530_03-02.csv f